In [127]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import joblib

## Load Dataset

In [128]:
dtypes = {
    'age': 'int', 
    'sex': 'str', 
    'cp': 'str', 
    'trestbps': 'int', 
    'chol': 'int', 
    'fbs': 'str', 
    'restecg': 'str', 
    'thalach': 'int', 
    'exang': 'str', 
    'oldpeak': 'float', 
    'slope': 'str', 
    'ca': 'str', 
    'thal': 'str', 
    'target': 'int',
}

df_data = pd.read_csv("../data/heart.xls", dtype=dtypes)
df_data.shape

(303, 14)

In [129]:
df_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Attribute Information:

Only 14 attributes used:

1. age: age in years (**numeric**)
2. sex: (1 = male; 0 = female) (**categorical**)
3. cp: chest pain type (**categorical**)
    - Value 1: typical angina
    - Value 2: atypical angina
    - Value 3: non-anginal pain
    - Value 4: asymptomatic 
4. trestbps: resting blood pressure (in mm Hg on admission to the hospital) (**numeric**)
5. chol: serum cholestoral in mg/dl (**numeric**)
6. fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) (**categorical**)
7. restecg: resting electrocardiographic results (**categorical**)
    - Value 0: normal
    - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria 
8. thalach: maximum heart rate achieved (**numeric**)
9. exang: exercise induced angina (1 = yes; 0 = no) (**categorical**)
10. oldpeak = ST depression induced by exercise relative to rest (**numeric**)
11. slope: the slope of the peak exercise ST segment (**categorical**)
    - Value 1: upsloping
    - Value 2: flat
    - Value 3: downsloping 
12. ca: number of major vessels (0-3) colored by flourosopy (**categorical**)
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect (**categorical**)
14. num: diagnosis of heart disease (angiographic disease status) (**categorical**) (the predicted attribute) 
    - Value 0: < 50% diameter narrowing
    - Value 1: > 50% diameter narrowing
    - (in any major vessel: attributes 59 through 68 are vessels)
    
    In short, 
    - 0 means absence of heart disease;
    - 1 means presence of heart disease;


In [130]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    object 
 2   cp        303 non-null    object 
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    object 
 6   restecg   303 non-null    object 
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    object 
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    object 
 11  ca        303 non-null    object 
 12  thal      303 non-null    object 
 13  target    303 non-null    int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 33.3+ KB


In [131]:
df_data["target"].value_counts()

1    165
0    138
Name: target, dtype: int64

In [132]:
X, y = df_data.drop(columns=["target"]), df_data[["target"]]
print(X.shape)
print(y.shape)

(303, 13)
(303, 1)


## Split dataset

In [133]:
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=123)

for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# reset the suffled indexes
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(212, 13)
(91, 13)
(212, 1)
(91, 1)


In [134]:
print(y_train.value_counts())
print(y_test.value_counts())

target
1         115
0          97
dtype: int64
target
1         50
0         41
dtype: int64


## Balance the Training Dataset

In [135]:
#sm = SMOTE(random_state=123)
#X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
#print(X_train_res.shape)
#print(y_train_res.shape)

In [136]:
cat_col = X_train.select_dtypes(include=['object']).columns.values
print(cat_col)

['sex' 'cp' 'fbs' 'restecg' 'exang' 'slope' 'ca' 'thal']


In [137]:
ohe = {
    c: OneHotEncoder(sparse=False, handle_unknown='ignore')
    for c in cat_col
}
ohe

{'sex': OneHotEncoder(handle_unknown='ignore', sparse=False),
 'cp': OneHotEncoder(handle_unknown='ignore', sparse=False),
 'fbs': OneHotEncoder(handle_unknown='ignore', sparse=False),
 'restecg': OneHotEncoder(handle_unknown='ignore', sparse=False),
 'exang': OneHotEncoder(handle_unknown='ignore', sparse=False),
 'slope': OneHotEncoder(handle_unknown='ignore', sparse=False),
 'ca': OneHotEncoder(handle_unknown='ignore', sparse=False),
 'thal': OneHotEncoder(handle_unknown='ignore', sparse=False)}

In [138]:
for c in cat_col:
    ohe[c].fit(X_train[c].values.reshape(-1, 1))
    print(f"{c}\t->\t{ohe[c].categories_}")

sex	->	[array(['0', '1'], dtype=object)]
cp	->	[array(['0', '1', '2', '3'], dtype=object)]
fbs	->	[array(['0', '1'], dtype=object)]
restecg	->	[array(['0', '1', '2'], dtype=object)]
exang	->	[array(['0', '1'], dtype=object)]
slope	->	[array(['0', '1', '2'], dtype=object)]
ca	->	[array(['0', '1', '2', '3', '4'], dtype=object)]
thal	->	[array(['0', '1', '2', '3'], dtype=object)]


In [139]:
X_transformed = None
for c in cat_col:
    feat_arr = ohe[c].transform(X_train[c].values.reshape(-1, 1))

    if X_transformed is None:
        X_transformed = pd.DataFrame(feat_arr)
        X_transformed.columns = [
            "{}_{}".format(c, v.replace(" ", "_"))
            for v in ohe[c].categories_[0]
        ]
    
    else:
        _X_buff = pd.DataFrame(feat_arr)
        _X_buff.columns = [
            f"{c}_{v.replace(' ', '_')}"
            for v in ohe[c].categories_[0]
        ]
        X_transformed = pd.concat([X_transformed, _X_buff], axis=1)

print(X_transformed.shape)

(212, 25)


In [140]:
X_transformed.head()

,sex_0,sex_1,cp_0,cp_1,cp_2,cp_3,fbs_0,fbs_1,restecg_0,restecg_1,...,slope_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## Concat numeric with one hot variables

In [143]:
# drop categorical columns to concat one hot
X_train = X_train.drop(columns=cat_col)

X_train = pd.concat([
    X_train, 
    X_transformed
], axis=1)

print(f"After concat:\t{X_train.shape}")

After concat:	(212, 30)


In [144]:
X_train.head()

,age,trestbps,chol,thalach,oldpeak,sex_0,sex_1,cp_0,cp_1,cp_2,...,slope_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
0,60,120,178,96,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,63,140,195,179,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,60,130,206,132,2.4,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,64,180,325,154,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,38,138,175,173,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## Apply one hot to test set

In [145]:
X_transformed = None
for c in cat_col:
    feat_arr = ohe[c].transform(X_test[c].values.reshape(-1, 1))

    if X_transformed is None:
        X_transformed = pd.DataFrame(feat_arr)
        X_transformed.columns = [
            "{}_{}".format(c, v.replace(" ", "_"))
            for v in ohe[c].categories_[0]
        ]
    
    else:
        _X_buff = pd.DataFrame(feat_arr)
        _X_buff.columns = [
            f"{c}_{v.replace(' ', '_')}"
            for v in ohe[c].categories_[0]
        ]
        X_transformed = pd.concat([X_transformed, _X_buff], axis=1)

## Concat One Hot with Numeric variables on test set

In [149]:
# drop categorical columns to concat one hot
X_test = X_test.drop(columns=cat_col)

X_test = pd.concat([
    X_test, 
    X_transformed
], axis=1)

print(f"After concat:\t{X_test.shape}")

After concat:	(91, 30)


## Build Model

In [150]:
modelXGBM = xgb.XGBClassifier(colsample_bytree= 1, learning_rate=0.1, max_depth=4, 
                              min_child_weight=1e-05, n_estimators=200, 
                              objective='binary:logistic', subsample=0.5)

In [151]:
print(X_train.shape)
print(y_train.shape)

(212, 30)
(212, 1)


In [152]:
modelXGBM.fit(X_train, y_train)
model_prediction = modelXGBM.predict(X_test)

[21:08:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/home/paulo/Documents/workspace/heart-disease-classifier/venv/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/paulo/Documents/workspace/heart-disease-classifier/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, 

In [153]:

print(accuracy_score(y_test, model_prediction))
print(roc_auc_score(y_test, model_prediction))

0.7582417582417582
0.7536585365853659


## Persist Model and data used

In [154]:
joblib.dump(modelXGBM, "../data/modelXGBM.pkl")

['../data/modelXGBM.pkl']

In [155]:
X_train_res.to_csv("../data/X_train.csv")
X_test.to_csv("../data/X_test.csv")
y_train_res.to_csv("../data/y_train.csv")
y_test.to_csv("../data/y_test.csv")